<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/D-Displaying_JSON_data_as_a_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://pyngrok.readthedocs.io/en/latest/integrations.html#colab-http-example
    

In [ ]:
https://digitalfox-tutorials.com/tutorial.php?title=Display-JSON-data-in-HTML-table-using-JavaScript

In [ ]:
https://www.w3docs.com/snippets/html/how-to-display-base64-images-in-html.html

In [ ]:
https://www.w3schools.com/jsref/prop_img_src.asp

In [ ]:
%html 

<body>
   <table>
      <thead>
         <tr>
            <th>ID</th>
            <th>Name</th>
            <th>Price</th>
            <th>Lat</th>
            <th>Lon</th>
         </tr>
      </thead>
      <tbody id="data-output">
         <!-- Prodcuts from javascript file in here. -->
      </tbody>
   </table>
 
<!-- The javascript code that calls the API-->
<script>

fetch("/citibike_api")
.then(function(response){
   return response.json();
})
.then(function(stations){
   let placeholder = document.querySelector("#data-output");
   let out = "";
   for(let s of stations.stations){
      out += `
         <tr>
            <td>${s.id}</td>
            <td>${s.name}</td>
            <td>${s.capacity}</td>
            <td>${s.lat}</td>
            <td>${s.lon}</td>
         </tr>
      `;
   }
   placeholder.innerHTML = out;
});

</script> 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os 
from flask import Flask, render_template, jsonify
from pyngrok import ngrok

from sqlalchemy import create_engine
import pandas as pd

os.environ["FLASK_DEBUG"] = "true"

app = Flask(__name__,  template_folder = '/content/drive/My Drive/templates')
port = 5000

# Open a ngrok tunnel to the HTTP server
ngrok_authtoken = '2EYf3qVk9mi739HjPwSNZXWAtfy_4jF9NAhGqVVVJmm4YehPW'
ngrok.set_auth_token(ngrok_authtoken)
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url
 

# Setup a connection to the database
conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')
engine = create_engine(conn_string)

@app.route('/citibike_api',  methods=['GET'])
def citibike_stations():

    sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
    # Connect to the database, execute the query, and get back the results
    with engine.connect() as connection:
        stations = pd.read_sql(sql, con=connection)
    
    # Create the response. We will put the retrieved data as a list of
    # dictionaries, under the key "stations".
    list_of_stations = stations.to_dict(orient='records')
    
    api_results = {"stations": list_of_stations}

    # We JSON-ify our dictionary and return it as the API response
    return jsonify(api_results)

 
app.run(use_reloader=False)